# Example Raspy Usage

This Notebook covers some examples of how to use Raspy with a HEC-RAS project.  It will demonstrate:

- Retrieving flow data
- Setting roughness
- Changing flow files

## General Setup

First, set up a `Ras` object pointed to your project.  This provides direct control over HEC-RAS.  Note that the project should be set to the plan you want to use.  The demo project I use is from HEC-RAS 5.0.7, but you can specify a different version using, e.g., `which="631"` for HEC-RAS 6.4.1.  (I have run some basic tests with HEC-RAS 6, but not extensively.  You may encounter bugs.)

In [1]:
from raspy_auto import *

In [3]:
# Update with your own project path.
project_path = r"C:\Users\dphilippus\Dropbox\Home\Research\RaspyDev\raspy_auto\TestProj\test.prj"
# Set up a HEC-RAS control object
ras = Ras(projectPath=project_path, which="507")

The `Ras` object provides direct, lower-level HEC-RAS control.  It also loads in, and can be used to examine, the model structure (e.g., rivers, reaches).  For example, below I will list all rivers, reaches, and cross-sections in the demo project.  Behold my creativity in naming rivers and reaches.

In [18]:
ras.rivers
r = ras.rivers[0].reaches[1]
r.getCrossSections()
r.xses[0].rs

for river in ras.rivers:
    for reach in river.reaches:
        for xs in reach.xses:
            print(f"{river.river} {reach.reach} {xs.rs}")

RiverOne Upper 2500
RiverOne Upper 2400
RiverOne Upper 2300
RiverOne Upper 2200
RiverOne Upper 2100
RiverOne Lower 2000
RiverOne Lower 1000
RiverOne Lower 601
RiverOne Lower 501
RiverOne Lower 401
RiverOne Lower 301
RiverOne Lower 201
RiverOne Lower 101
RiverOne Lower 1
Thingy OnlyOne 500
Thingy OnlyOne 400
Thingy OnlyOne 300
Thingy OnlyOne 200
Thingy OnlyOne 100


## API

If we want to control a HEC-RAS project more conveniently with high-level functions, we should instead use the `API` class.  This is generally the practical tool for Raspy usage, and builds on the lower-level capabilities in `Ras` objects.  (Note that this means different implementations of direct HEC-RAS control can be provided: any `Ras` object with the required methods will work.)

In [19]:
api = API(ras)

The `API` object provides several groups of methods, under `data`, `ops`, and `params`.  Data methods are used to retrieve model data.  Params methods set model parameters (e.g., roughness).  Ops methods handle model operations, like running.

Note that Raspy's `params` methods focus on straightforward tasks like changing roughness and flow.  In-depth geometry modifications require directly editing geometry files, which is implemented by the separate `RaspyGeo` package.

### Retrieving Data

The `API.data` object has methods for retrieving velocity, stage, and shear for everything, a whole river, all cross-sections in a reach, or an individual cross-section.  It does this for a specified number of flow profiles (the user must specify the number, since the HEC-RAS COM does not provide a way to list the number of profiles).  You can also retrieve full flow data, either for the main channel only (`api.data.allFlow()`) or the left overbank, main channel, and right overbank (`api.data.allFlowDist()`).  Likewise, the methods to retrieve velocity, etc have a variant ending in `Dist` that retrieves the distribution across the channel.  If you retrieve data for multiple rivers, reaches, cross-sections, or profiles, the result will be a nested dictionary.

In [65]:
api.data.stage('RiverOne', 'Lower', '401', 10)

{1: 1.0122871398925781,
 2: 2.142230987548828,
 3: 2.908954620361328,
 4: 4.2667083740234375,
 5: 6.135383605957031,
 6: 8.291099548339844,
 7: 11.470245361328125,
 8: 12.80145263671875,
 9: 14.032791137695312,
 10: 15.175979614257812}

In [64]:
data = api.data.allFlowDist()
{rivn: {
    rchn: {
        rs: {
            "Q": xs.flow, "v": xs.velocity, "y": xs.maxDepth, "shear": xs.shear, "etc": xs.etc
        } for rs, xs in rch.items()
    } for rchn, rch in riv.items()
} for rivn, riv in data.items()}

{'RiverOne': {'Upper': {'2500': {'Q': [0, 10.0, 0],
    'v': [0, 4.871948719024658, 0],
    'y': [0, 1.43267822265625, 0],
    'shear': [0, 0.19032390415668488, 0],
    'etc': {'area': [0, 4.109661102294922, 0],
     'wp': [0, 2.0525667667388916, 0]}},
   '2400': {'Q': [0, 10.0, 0],
    'v': [0, 4.871948719024658, 0],
    'y': [0, 1.43267822265625, 0],
    'shear': [0, 0.19032390415668488, 0],
    'etc': {'area': [0, 4.109661102294922, 0],
     'wp': [0, 2.0525667667388916, 0]}},
   '2300': {'Q': [0, 10.0, 0],
    'v': [0, 4.871948719024658, 0],
    'y': [0, 1.43267822265625, 0],
    'shear': [0, 0.19032390415668488, 0],
    'etc': {'area': [0, 4.109661102294922, 0],
     'wp': [0, 2.0525667667388916, 0]}},
   '2200': {'Q': [0, 10.0, 0],
    'v': [0, 4.871948719024658, 0],
    'y': [0, 1.43267822265625, 0],
    'shear': [0, 0.19032390415668488, 0],
    'etc': {'area': [0, 4.109661102294922, 0],
     'wp': [0, 2.0525667667388916, 0]}},
   '2100': {'Q': [0, 10.0, 0],
    'v': [0, 4.87132

### Setting Roughness and Running the Model

Raspy supports setting roughness for whole reaches or individual cross-sections, and either for an entire cross-section or left/main/right.  To specify for individual cross-sections, the `manning` argument should be a dictionary of `{"river station": n}`.  To set multiple roughnesses within a cross-section, provide `n` as a list rather than a single value (e.g., `[0.15, 0.017, 0.15]` for a concrete channel with willow-covered overbanks).  In this demo, I compare stage results before and after setting roughness.

In [66]:
for river in ras.rivers:
    for reach in river.reaches:
        for xs in reach.xses:
            print(f"{river.river} {reach.reach} {xs.rs}")

RiverOne Upper 2500
RiverOne Upper 2400
RiverOne Upper 2300
RiverOne Upper 2200
RiverOne Upper 2100
RiverOne Lower 2000
RiverOne Lower 1000
RiverOne Lower 601
RiverOne Lower 501
RiverOne Lower 401
RiverOne Lower 301
RiverOne Lower 201
RiverOne Lower 101
RiverOne Lower 1
Thingy OnlyOne 500
Thingy OnlyOne 400
Thingy OnlyOne 300
Thingy OnlyOne 200
Thingy OnlyOne 100


In [69]:
api.params.modifyN(river="RiverOne", reach="Lower", manning = 0.017)
api.ops.compute()
api.data.stage("RiverOne", "Lower", "1000", 10)

{1: 0.3905487060546875,
 2: 1.0359344482421875,
 3: 1.5390701293945312,
 4: 2.6072845458984375,
 5: 4.058616638183594,
 6: 5.2050628662109375,
 7: 6.9441986083984375,
 8: 7.722938537597656,
 9: 8.423637390136719,
 10: 9.069602966308594}

In [70]:
api.params.modifyN(river="RiverOne", reach="Lower", manning = {"2000": 0.1, "1000": [0.1, 0.03, 0.1], "601": 0.03, "501": 0.03, "401": 0.03, "301": 0.03})
api.ops.compute()
api.data.stage("RiverOne", "Lower", "1000", 10)

{1: 0.3905487060546875,
 2: 1.0359344482421875,
 3: 1.5390701293945312,
 4: 2.6072845458984375,
 5: 4.094703674316406,
 6: 5.572822570800781,
 7: 7.814453125,
 8: 8.819190979003906,
 9: 9.6995849609375,
 10: 10.577751159667969}

### Setting Flow Profiles

Raspy supports specifying new flow profiles.  You must provide an existing flow file which the current plan is set up to use, and which will be overwritten (because it was much more convenient to do it that way than to edit the plan to point to a new flow file).  This functionality directly modifies the file, since HEC-RAS' function for setting flow doesn't work well if the number of profiles change (or didn't, in 5.0.7).  The file number is just the numeric part of the file extention, so `"01"` for `project.f01`.  (I have not tested setting a change of discharge mid-reach; it should work, but it may be necessary to manually insert discharge change points first.)

In [71]:
api.params.setSteadyFlows(fileN="01", river="RiverOne", reach="Lower", rs="2000", flows=list(range(100, 5100, 100)))
api.ops.compute()
api.data.stage("RiverOne", "Lower", "1000", 50)  # there are now 50 profiles

{1: 1.223968505859375,
 2: 1.8032455444335938,
 3: 2.2424163818359375,
 4: 2.6072235107421875,
 5: 2.9239349365234375,
 6: 3.2642059326171875,
 7: 3.498321533203125,
 8: 3.7131423950195312,
 9: 3.8961868286132812,
 10: 4.094703674316406,
 11: 4.266777038574219,
 12: 4.425994873046875,
 13: 4.5731353759765625,
 14: 4.7230682373046875,
 15: 4.882865905761719,
 16: 5.0311431884765625,
 17: 5.1745147705078125,
 18: 5.312690734863281,
 19: 5.4454803466796875,
 20: 5.572822570800781,
 21: 5.6840667724609375,
 22: 5.814964294433594,
 23: 5.9361419677734375,
 24: 6.0567169189453125,
 25: 6.1919708251953125,
 26: 6.314079284667969,
 27: 6.434173583984375,
 28: 6.552055358886719,
 29: 6.667640686035156,
 30: 6.780891418457031,
 31: 6.8917236328125,
 32: 7.000007629394531,
 33: 7.105804443359375,
 34: 7.209007263183594,
 35: 7.297950744628906,
 36: 7.4064178466796875,
 37: 7.514442443847656,
 38: 7.612770080566406,
 39: 7.714134216308594,
 40: 7.814453125,
 41: 7.931793212890625,
 42: 8.034370422